In [5]:
import numpy as np
from tqdm import tqdm
from solver.qp import qp
import torch
import gzip
import pickle
import warnings
from generate_instances import generate_softmarginsvm

### Soft margin Support Vector Machine

In [6]:
def surrogate_gen():
    C = 10 #regularization parameter
    m = 4 #number of Datapoints
    X = np.random.uniform(-1, 1, (m, 2)) #Dataset
    bias = np.full((m, 1), 0.5)
    X[:, 1] += bias.ravel()   # Adding a bias to make data linearly separable
    y = np.array([1.0 if x[0] > x[1] else -1.0 for x in X]).reshape(m, 1) #label vector
    X_dash = (y.reshape(-1,1) * 1.) * X #X_dash * X_dash^T = Q
    Q, q, G, h, A, b = generate_softmarginsvm(y, X, X_dash, C)    
    return Q, q, G, h, A, b, X_dash

### create

In [29]:
import os

directory = f'Quadratic_Programming_Datasets/raw'
os.makedirs(directory, exist_ok=True)
warnings.filterwarnings("error")

ips = []
pkg_idx = 0 #identifier index
success_cnt = 0
fail_cnt = 0
max_ipm_steps = 0


max_iter = 100
num = 10

for i in tqdm(range(max_iter)):
    Q, q, G, h, A, b, S = surrogate_gen()
    
    try:
        res = qp(Q, q, G, h, A, b, callback=lambda res: res)
        max_ipm_steps = max(max_ipm_steps, res['iterations'])
        print("status",res['status'])
    except Exception as e:
        fail_cnt += 1
        warnings.warn(f'Optimization failed with error: {str(e)}')
        continue
    else:
        if res['status'] == 'optimal': #np.isnan(res.fun)?
            ips.append((torch.from_numpy(np.array(Q)).to(torch.float), torch.from_numpy(np.array(q)).to(torch.float), torch.from_numpy(np.array(G)).to(torch.float), torch.from_numpy(np.array(h)).to(torch.float), torch.from_numpy(np.array(A)).to(torch.float), torch.from_numpy(np.array(b)).to(torch.float), torch.from_numpy(np.array(S)).to(torch.float), res))
            print(success_cnt)
            success_cnt += 1
            
    if len(ips) >= 1000 or success_cnt == num:
        print("success")
        ips = [ip + (max_ipm_steps,) for ip in ips]
        with gzip.open(f'Quadratic_Programming_Datasets/raw/instance_{pkg_idx}.pkl.gz', "wb") as file:
            pickle.dump(ips, file)
            print(f'Package {pkg_idx} saved')
            pkg_idx += 1

        ips = []
    if success_cnt >= num:
        break
warnings.resetwarnings()

  9%|▉         | 9/100 [00:00<00:02, 40.64it/s]

     pcost       dcost       gap    pres   dres
 0: -4.9198e-02 -1.2479e+02  1e+02  4e-16  1e-15
 1: -9.3396e+00 -2.0742e+01  1e+01  9e-16  9e-16
 2: -1.4638e+01 -1.6217e+01  2e+00  1e-16  2e-16
 3: -1.4998e+01 -1.5084e+01  9e-02  2e-15  2e-16
 4: -1.5023e+01 -1.5025e+01  1e-03  1e-15  4e-16
 5: -1.5024e+01 -1.5024e+01  1e-05  4e-16  4e-16
Optimal solution found.
status optimal
0
     pcost       dcost       gap    pres   dres
 0:  1.8673e+01 -9.6094e+01  1e+02  3e-17  1e-15
 1:  2.4330e+00 -9.3512e+00  1e+01  1e-15  2e-15
 2: -7.6971e-01 -2.2158e+00  1e+00  2e-16  5e-16
 3: -1.0822e+00 -1.1878e+00  1e-01  2e-16  3e-16
 4: -1.1482e+00 -1.1565e+00  8e-03  1e-16  3e-16
 5: -1.1528e+00 -1.1537e+00  9e-04  1e-16  3e-16
 6: -1.1535e+00 -1.1535e+00  4e-05  2e-16  7e-17
 7: -1.1535e+00 -1.1535e+00  4e-07  1e-16  2e-16
Optimal solution found.
status optimal
1
     pcost       dcost       gap    pres   dres
 0:  4.9520e-16 -4.0000e+01  9e+01  1e-01  0e+00
 1:  7.1797e-17 -7.3007e-01  1e+00  1e-

In [8]:
# Open and load the .pkl.gz file
with gzip.open(f'Quadratic_Programming_Datasets/raw/instance_{pkg_idx-1}.pkl.gz', 'rb') as f:
    data = pickle.load(f)

# Now `data` contains the contents of the pickle file
print(data)

[(tensor([[1.0531, 0.4732, 0.9198, 0.3978],
        [0.4732, 1.1517, 0.1529, 0.2037],
        [0.9198, 0.1529, 0.8756, 0.3406],
        [0.3978, 0.2037, 0.3406, 0.1510]]), tensor([[-1.],
        [-1.],
        [-1.],
        [-1.]]), tensor([[-1., -0., -0., -0.],
        [-0., -1., -0., -0.],
        [-0., -0., -1., -0.],
        [-0., -0., -0., -1.],
        [ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]), tensor([[ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [10.],
        [10.],
        [10.],
        [10.]]), tensor([[ 1., -1.,  1., -1.]]), tensor([[0.]]), tensor([[ 0.9648, -0.3497],
        [ 0.1033, -1.0682],
        [ 0.9342, -0.0528],
        [ 0.3557, -0.1563]])), (tensor([[ 1.0200, -0.7671,  0.9404,  0.5987],
        [-0.7671,  2.0544, -0.9839, -0.3177],
        [ 0.9404, -0.9839,  0.9189,  0.5272],
        [ 0.5987, -0.3177,  0.5272,  0.3633]]), tensor([[-1.],
        [-1.],
        [-1.],
       